## **SpDB Manual之系列3：Entry进阶——Open_Entry**

正如上一个系列2中介绍的，SpDB中引入Entry思想来灵活处理Python中的原生数据类型。
这种概念对于大型数据处理非常有意义，因为它可以根据需要灵活的进行根节点指针的游走，获取对应的节点的数据，而无需获取整个根目录下的数据。

日常的科研环境中，数据科学的类似是多样的，通常被分类为：
- 结构化数据
- 半结构化数据
- 非结构化数据

这些数据类型以不同的形式表现出来，如结构化的数据通常以表格的形式出现，
半结构化的数据通常以XML或者JSON的形式出现，非结构化的数据通常以二进制的形式出现，如磁约束聚变中常用的GDSKfile。

SpDB通过对底层Entry功能封装，提供统一的Open_Entry标准API来访问、处理这些数据。Open_Entry中支持标准的URL语法来访问不同的数据。用户无需关心后台的数据格式及来源。



标准的URI表述,它是标准RFC3986的扩展：
```
<schema>+<protocol>+<backend>://< authority >/<path>/?<query>#<fragment>
```
- \<schema\> 则指定了访问资源的 "语义"。
- \<protocol\>指定了资源的协议，如远程的MDSplus或者本地的数据文件格式
- \<backend\>：指定了访问资源的 "格式；
- \<authority\> 组件指定了管理资源的权限，通常包括主机 通常包括主机名和端口号。
- \<path\>组件指定资源的分层路径，可包括多个层级，每个层级之间用 资源的分层路径，可包括多个层级，以斜线（/）分隔。
- \<query\> 组件指定随资源请求发送到服务器的查询字符串
- \<fragment\> 组件指定指向特定资源的 片段标识符

标准的URI表述,它是标准RFC3986的扩展：
```
<device>+<protocol>://<netloc>/<path>?<query>#<fragment>

```
- device: specifies  device description;
- \<protocol\>://\<netloc\>/\<path\>: specifies the address of the data source
- \<query\> query command, e.g. specify shot or  run number
- \<fragment\>specifies  path to subnodes


####处理文件的URL表达式：
```
file_entry = open_entry(f"file+format://{file_path}}")
```
- open_entry() :API入口
- file：处理对象是文件
- format：指定文件格式
- file_path：文件路径

In [102]:
from spdm.data.Entry import open_entry, Entry, EntryProxy
from spdm.data.File import File
from spdm.utils.logger import logger

file_entry = open_entry(f"file+geqdsk://{workdir}//data/g900003.00230_ITER_15MA_eqdsk16HR.txt/")
### file表示处理对象是文件
### geqdsk表示文件类型是geqdsk
### 

### 指定子节点的标识符
file_entry = open_entry(f"file+geqdsk://{workdir}//data/g900003.00230_ITER_15MA_eqdsk16HR.txt/#equilibrium")


#### 处理数据库系统的URL表达式：
```

db_entry = open_entry(f"db+protocol://{db_path}}")
entry = open_entry("device+mdsplus://202.127.204.12?enable=efit_east&shot={shot_num}")

entry_local = open_entry(f"east+mdsplus://{DATA_PATH}/mdsplus/~t/?shot=70745")

```
- device : 指定数据来源的装置
- mdsplus： 处理对象是MDS数据库
- 202.127.204.12 ：EAST mds数据库访问地址
- enable：同时可以获取efit_east数据
- shot：指定炮号

怎么阐述这些？？？


In [103]:
[*file_entry.keys()]

['wall', 'equilibrium']

In [63]:
## 加载基本环境
from spdm.data.Entry import open_entry, Entry, EntryProxy
from spdm.data.File import File
from spdm.utils.logger import logger
import os
### 需要解决，暂时先依赖fytok，理论上spdb单独使用不需要fytok
os.environ["SP_DATA_MAPPING_PATH"] = "/gpfs/fuyun//projects/fuyun/fytok/python/fytok/_mapping/"

In [74]:
### 指定workdir
workdir = "/scratch/jupytertest/workspace_fytok/fytok_tutorial/tutorial/"


In [75]:
## 通过open_entry函数,将原始换的gdskfile，转成entry对象，B02_SpDB_entry中对entry的操作均有效

file_entry = open_entry(f"file+geqdsk://{workdir}//data/g900003.00230_ITER_15MA_eqdsk16HR.txt/#equilibrium")

# /scratch/jupytertest/workspace_fytok/fytok_tutorial/tutorial/data/g900003.00230_ITER_15MA_eqdsk16HR.txt

In [ ]:
### 访问远程EAST MDS数据库中70754炮的数据，
shot_num = 70754
time_slice = 10
entry = open_entry("east+mdsplus://202.127.204.12?enable=efit_east&shot={shot_num}")

In [76]:
[*file_entry.keys()]

['time', 'vacuum_toroidal_field', 'time_slice']

In [77]:
file_entry.get("vacuum_toroidal_field")

{'r0': 6.2, 'b0': [-5.3]}

In [78]:
file_entry.child("vacuum_toroidal_field").fetch()

{'r0': 6.2, 'b0': [-5.3]}

#### "hdf5_entry 要确定下。"

In [79]:
## 通过open_entry函数,将原始换的gdskfile，转成entry对象，B02_SpDB_entry中对entry的操作均有效
hdf5_entry = open_entry(f"file+hdf5://{workdir}/data/test_eq.h5")

2023-11-15 20:17:10,780 [    spdm]    DEBUG: /gpfs/fuyun/projects/fuyun/spdm/python/spdm/utils/sp_export.py:66:sp_load_module: Load module spdm.plugins.data.plugin_hdf5
2023-11-15 20:17:10,793 [    spdm]    DEBUG: /gpfs/fuyun/projects/fuyun/spdm/python/spdm/plugins/data/plugin_hdf5.py:196:open: Open HDF5 File /scratch/jupytertest/workspace_fytok/fytok_tutorial/tutorial//data/test_eq.h5 mode=Mode.read


In [82]:
[*hdf5_entry.keys()]

['equilibrium', 'wall']

In [97]:
hdf5_entry.get("wall.description_2d[0].limiter.unit[0].outline.r")

array([1.35838, 1.35838, 1.35838, 1.36314, 1.4371 , 1.43721, 1.44164,
       1.4297 , 1.39169, 1.39151, 1.37929, 1.399  , 1.43623, 1.45919,
       1.47791, 1.54494, 1.61204, 1.63506, 1.66054, 1.66519, 1.6955 ,
       1.70668, 1.71388, 1.714  , 1.73649, 1.76324, 1.80199, 1.80237,
       1.93972, 1.97063, 2.07495, 2.1771 , 2.27925, 2.35   , 2.35   ,
       2.35   , 2.27925, 2.1771 , 2.07495, 1.9728 , 1.97063, 1.92059,
       1.87055, 1.82051, 1.79281, 1.76511, 1.75421, 1.67782, 1.60143,
       1.48426, 1.36709, 1.33132, 1.37874, 1.42615, 1.40528, 1.3844 ,
       1.36353, 1.35838, 1.35838, 1.35838, 1.35838])

In [101]:
hdf5_entry.child("wall").fetch()

{'description_2d': [{'limiter': {'unit': [{'outline': {'r': array([1.35838, 1.35838, 1.35838, 1.36314, 1.4371 , 1.43721, 1.44164,
              1.4297 , 1.39169, 1.39151, 1.37929, 1.399  , 1.43623, 1.45919,
              1.47791, 1.54494, 1.61204, 1.63506, 1.66054, 1.66519, 1.6955 ,
              1.70668, 1.71388, 1.714  , 1.73649, 1.76324, 1.80199, 1.80237,
              1.93972, 1.97063, 2.07495, 2.1771 , 2.27925, 2.35   , 2.35   ,
              2.35   , 2.27925, 2.1771 , 2.07495, 1.9728 , 1.97063, 1.92059,
              1.87055, 1.82051, 1.79281, 1.76511, 1.75421, 1.67782, 1.60143,
              1.48426, 1.36709, 1.33132, 1.37874, 1.42615, 1.40528, 1.3844 ,
              1.36353, 1.35838, 1.35838, 1.35838, 1.35838]),
       'z': array([ 0.      ,  0.227   ,  0.454   ,  0.455735,  0.798332,  0.798821,
               0.838195,  0.908908,  1.01426 ,  1.01473 ,  1.04859 ,  1.03946 ,
               1.0256  ,  1.02095 ,  1.02514 ,  1.05454 ,  1.08394 ,  1.09586 ,
               1.114   , 

In [100]:
hdf5_entry.child("wall.description_2d[0].limiter").fetch()

{'unit': [{'outline': {'r': array([1.35838, 1.35838, 1.35838, 1.36314, 1.4371 , 1.43721, 1.44164,
           1.4297 , 1.39169, 1.39151, 1.37929, 1.399  , 1.43623, 1.45919,
           1.47791, 1.54494, 1.61204, 1.63506, 1.66054, 1.66519, 1.6955 ,
           1.70668, 1.71388, 1.714  , 1.73649, 1.76324, 1.80199, 1.80237,
           1.93972, 1.97063, 2.07495, 2.1771 , 2.27925, 2.35   , 2.35   ,
           2.35   , 2.27925, 2.1771 , 2.07495, 1.9728 , 1.97063, 1.92059,
           1.87055, 1.82051, 1.79281, 1.76511, 1.75421, 1.67782, 1.60143,
           1.48426, 1.36709, 1.33132, 1.37874, 1.42615, 1.40528, 1.3844 ,
           1.36353, 1.35838, 1.35838, 1.35838, 1.35838]),
    'z': array([ 0.      ,  0.227   ,  0.454   ,  0.455735,  0.798332,  0.798821,
            0.838195,  0.908908,  1.01426 ,  1.01473 ,  1.04859 ,  1.03946 ,
            1.0256  ,  1.02095 ,  1.02514 ,  1.05454 ,  1.08394 ,  1.09586 ,
            1.114   ,  1.11765 ,  1.14183 ,  1.16211 ,  1.13093 ,  1.13044 ,
            1

In [84]:
hdf5_entry.get("equilibrium/time_slice/0/profiles_2d/psi") 

array([[-0.41393183, -0.40004918, -0.38699915, ..., -0.40942308,
        -0.42310933, -0.43761767],
       [-0.41614134, -0.40232849, -0.38933536, ..., -0.41150394,
        -0.42512211, -0.43954969],
       [-0.41808789, -0.40436263, -0.39144252, ..., -0.41335733,
        -0.42688992, -0.44121735],
       ...,
       [ 0.33574223,  0.31507868,  0.29482098, ...,  0.28401798,
         0.30266326,  0.32155843],
       [ 0.34931039,  0.3273043 ,  0.30587716, ...,  0.29560275,
         0.31543752,  0.33569433],
       [ 0.3628118 ,  0.33939058,  0.3167504 , ...,  0.30705014,
         0.32813432,  0.34984719]])

In [8]:
with File("./data/g070754.05000", mode="r", format="GEQdsk") as fid:
    doc = fid.read()
    eq_test = doc.dump()

####处理MDSplus的URL表达式

In [10]:
### 访问远程EAST MDS数据库中70754炮的数据，
shot_num = 70754
time_slice = 10
entry = open_entry("east+mdsplus://202.127.204.12?enable=efit_east&shot={shot_num}")

2023-11-15 14:46:56,152 [    spdm]    DEBUG: /home/salmon/workspace/fytok/SpDM/python/spdm/utils/sp_export.py:66:sp_load_module: Load module spdm.plugins.data.plugin_xml


In [11]:
entry.get('wall')

{'ids_properties': {'comment': {},
  'provider': 'Guo, Yong',
  'creation_date': '2020-10-12',
  'homogeneous_time': 2},
 'description_2d': {'type': {'name': 'equilibrium', 'index': 1},
  'limiter': {'type': {'name': 'limiter', 'index': 0},
   'unit': {'name': 'limiter',
    'closed': 1,
    'outline': {'r': array([2.277, 2.273, 2.267, 1.94 , 1.94 , 1.802, 1.773, 1.751, 1.736,
            1.714, 1.707, 1.696, 1.665, 1.656, 1.635, 1.612, 1.478, 1.459,
            1.44 , 1.436, 1.399, 1.379, 1.392, 1.43 , 1.439, 1.442, 1.437,
            1.363, 1.361, 1.361, 1.361, 1.363, 1.421, 1.423, 1.422, 1.418,
            1.331, 1.367, 1.564, 1.597, 1.598, 1.624, 1.754, 1.765, 1.814,
            1.824, 1.825, 1.841, 1.971, 1.971, 2.267, 2.273, 2.277, 2.277,
            2.306, 2.328, 2.343, 2.35 , 2.35 , 2.35 , 2.343, 2.328, 2.306,
            2.277]),
     'z': array([ 0.485,  0.485,  0.493,  0.809,  0.809,  0.926,  0.956,  0.993,
             1.033,  1.131,  1.162,  1.142,  1.117,  1.111,  1.096, 

In [12]:
entry.child('wall').fetch()

{'ids_properties': {'comment': {},
  'provider': 'Guo, Yong',
  'creation_date': '2020-10-12',
  'homogeneous_time': 2},
 'description_2d': {'type': {'name': 'equilibrium', 'index': 1},
  'limiter': {'type': {'name': 'limiter', 'index': 0},
   'unit': {'name': 'limiter',
    'closed': 1,
    'outline': {'r': array([2.277, 2.273, 2.267, 1.94 , 1.94 , 1.802, 1.773, 1.751, 1.736,
            1.714, 1.707, 1.696, 1.665, 1.656, 1.635, 1.612, 1.478, 1.459,
            1.44 , 1.436, 1.399, 1.379, 1.392, 1.43 , 1.439, 1.442, 1.437,
            1.363, 1.361, 1.361, 1.361, 1.363, 1.421, 1.423, 1.422, 1.418,
            1.331, 1.367, 1.564, 1.597, 1.598, 1.624, 1.754, 1.765, 1.814,
            1.824, 1.825, 1.841, 1.971, 1.971, 2.267, 2.273, 2.277, 2.277,
            2.306, 2.328, 2.343, 2.35 , 2.35 , 2.35 , 2.343, 2.328, 2.306,
            2.277]),
     'z': array([ 0.485,  0.485,  0.493,  0.809,  0.809,  0.926,  0.956,  0.993,
             1.033,  1.131,  1.162,  1.142,  1.117,  1.111,  1.096, 